In [1]:
from azureml.core import Workspace, Datastore, Dataset

from torch import nn
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

import matplotlib.pyplot as plt

from torch import optim

In [2]:
ws = Workspace.from_config()

ds = Datastore.get(ws, "cvdatastore")

dataset = Dataset.File.from_files(path=(ds, "Plant_leaf_diseases_dataset_with_augmentation/Plant_leave_diseases_dataset_with_augmentation/**"))

mount_context = dataset.mount()
mount_context.start()

data_path = mount_context.mount_point

In [3]:
# As the data is already augmented we will go with no aug transform
transform = transforms.Compose([
    transforms.Resize((244,244)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=data_path,transform=transform)

train_ratio = int(0.8*len(full_dataset))
test_ratio = len(full_dataset)-train_ratio

train_ds, test_ds = random_split(full_dataset, [train_ratio, test_ratio])


train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Model & eval definition

model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad=False

model.fc = nn.Linear(model.fc.in_features, len(full_dataset.classes))

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Training 
epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        out = model(images)
        loss = criterion(out, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device),labels.to(device)
            out = model(images)
            
            _, pred = torch.max(out,1)
            total += labels.size(0)
            correct += (pred==labels).sum().item()

    print(f"Epoch:{epoch}/{epochs}\tLoss:{running_loss}\tAccuracy:{100*correct/total}")

Epoch:0/5	Loss:157.0681322813034	Accuracy:94.31614896731176
Epoch:1/5	Loss:81.9086384922266	Accuracy:95.34070580582208
Epoch:2/5	Loss:62.601786985993385	Accuracy:95.9830866807611
Epoch:3/5	Loss:51.67556782066822	Accuracy:96.13758334688568
Epoch:4/5	Loss:45.24557711556554	Accuracy:96.52789071393723


In [8]:
mount_context.stop()

model.eval()
torch.save(model.state_dict,"leaf_disease_model.pth")